In [1]:
# Importación de librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing import image

# 1. Preparación de los datos
# Ruta a los conjuntos de datos de imágenes
# Aquí se debe especificar la ubicación del dataset. Este dataset debe estar organizado en subcarpetas
# donde cada subcarpeta representa una clase (por ejemplo, /dogs y /cats).
data_dir = "dataset_path"  # Reemplaza con la ruta a tu dataset

# Generador de datos con aumentación
# ImageDataGenerator permite realizar aumentación de datos para prevenir el sobreajuste.
# Rescale normaliza los valores de píxeles al rango [0, 1].
# Otros parámetros como rotation_range, zoom_range y horizontal_flip aplican transformaciones aleatorias.
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Escala los valores de los píxeles al rango [0, 1]
    rotation_range=20,  # Rotación aleatoria en un rango de 20 grados
    width_shift_range=0.2,  # Desplazamiento horizontal
    height_shift_range=0.2,  # Desplazamiento vertical
    shear_range=0.2,  # Transformación en cizalla
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    validation_split=0.2  # Divide los datos en 80% entrenamiento y 20% validación
)

# Carga de datos de entrenamiento
# Carga imágenes del directorio especificado y las convierte en lotes con las transformaciones definidas.
train_data = train_datagen.flow_from_directory(
    data_dir,  # Directorio raíz de los datos
    target_size=(150, 150),  # Redimensiona todas las imágenes a 150x150 píxeles
    batch_size=32,  # Tamaño del lote
    class_mode='binary',  # Tarea de clasificación binaria (perro/gato)
    subset='training'  # Subconjunto de entrenamiento
)

# Carga de datos de validación
# Similar a train_data, pero se usa el subconjunto de validación.
validation_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# 2. Diseño del modelo CNN
# Se crea una red neuronal convolucional (CNN) con 3 bloques de capas convolucionales y de agrupación.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),  # 32 filtros de 3x3, activación ReLU
    MaxPooling2D(pool_size=(2, 2)),  # Agrupación máxima 2x2 para reducir dimensiones

    Conv2D(64, (3, 3), activation='relu'),  # Segundo bloque convolucional
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),  # Tercer bloque convolucional
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),  # Aplana el tensor 3D en un vector 1D para las capas densas
    Dense(128, activation='relu'),  # Capa completamente conectada con 128 neuronas
    Dropout(0.5),  # Apaga aleatoriamente el 50% de las neuronas para prevenir sobreajuste
    Dense(1, activation='sigmoid')  # Capa de salida con activación sigmoidal (probabilidad)
])

# 3. Compilación del modelo
# Se configura el modelo con un optimizador (Adam), una función de pérdida adecuada y una métrica.
model.compile(
    optimizer='adam',  # Optimizador Adam
    loss='binary_crossentropy',  # Función de pérdida para clasificación binaria
    metrics=['accuracy']  # Métrica de precisión
)

# 4. Entrenamiento del modelo
# EarlyStopping detiene el entrenamiento si la pérdida de validación no mejora durante 5 épocas consecutivas.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamos el modelo con los datos de entrenamiento y validación.
history = model.fit(
    train_data,  # Datos de entrenamiento
    validation_data=validation_data,  # Datos de validación
    epochs=20,  # Número máximo de épocas
    callbacks=[early_stopping]  # Callback para detener entrenamiento temprano
)

# 5. Evaluación del modelo
# Evalúa el modelo en el conjunto de validación y muestra la precisión.
eval_results = model.evaluate(validation_data)
print(f"\nPrecisión en validación: {eval_results[1] * 100:.2f}%")

# 6. Guardar el modelo
# Guarda el modelo entrenado en un archivo .h5 para reutilizarlo posteriormente.
model.save("cnn_model.h5")

# 7. Cargar el modelo para predicciones
# Se carga el modelo previamente guardado.
loaded_model = tf.keras.models.load_model("cnn_model.h5")

# Predicción de nuevas imágenes
# Esta función permite cargar una imagen y predecir si es un perro o un gato.
def predict_image(img_path):
    # Carga la imagen y la redimensiona al tamaño requerido por el modelo
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0  # Normaliza la imagen
    img_array = np.expand_dims(img_array, axis=0)  # Agrega una dimensión para el lote

    # Realiza la predicción
    prediction = loaded_model.predict(img_array)
    if prediction[0] > 0.5:
        print(f"La imagen es un gato (Confianza: {prediction[0][0] * 100:.2f}%)")
    else:
        print(f"La imagen es un perro (Confianza: {(1 - prediction[0][0]) * 100:.2f}%)")

# Ejemplo de uso
# Llama a la función predict_image pasando la ruta de una nueva imagen.
predict_image("path_to_new_image.jpg")  # Reemplaza con la ruta de la imagen para predecir


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_path'